In [ ]:
# Crafting an AI-Powered HR Assistant: A Use Case for Nestle’s HR Policy Documents

#
# Install python libraries
#

! pip install langchain
! pip install langchain_community
! pip install langchain_openai
! pip install python-dotenv
! pip install streamlit
! pip install gradio
! pip install langchain_experimental
! pip install sentence-transformers
! pip install langchain_chroma
! pip install langchainhub
! pip install unstructured

! pip install langchain
! pip install langchain_openai
! pip install langchain_chroma
! pip install langchain_core
! pip install faiss-cpu 


In [3]:
#
# Load pdf file: the_nestle_hr_policy_pdf_2012
#
# The Nestlé Human Resources Policy
#
# Path is /voc/work

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/voc/work/the_nestle_hr_policy_pdf_2012.pdf")
data = loader.load()  # entire PDF is loaded as a single Document

print("File loaded successfully..")
print("\t\tthe_nestle_hr_policy_pdf_2012.pdf")

print("\nTotal Pages", len(data))
print()


File loaded successfully..
		the_nestle_hr_policy_pdf_2012.pdf

Total Pages 8



In [4]:
#
# Split the document 
#

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Document splitted successfully..")
print("\nTotal number splitted documents: ",len(docs))
print()


Document splitted successfully..

Total number splitted documents:  20



In [5]:
#
# Store the document in vectore database 
# 
# Add retriever based on similarity 
#

from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()


# Extract plain text from documents
texts = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

# Create FAISS vectorstore
vectorstore = FAISS.from_texts(texts, embedding=OpenAIEmbeddings(model="text-embedding-3-small"), metadatas=metadatas)
print("Vector store created using FAISS and OpenAI Embeddings")

# Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Query prompt
prompt = "What kind of document is it?"

print("Retrieve document, by prompt:\n\t\t", prompt)
retrieved_docs = retriever.invoke(prompt)

print("\nTotal retrieved documents:", len(retrieved_docs))
if retrieved_docs:
    print("\nCheck retrieved document[0]:\n", retrieved_docs[0].page_content)


Vector store created using FAISS and OpenAI Embeddings
Retrieve document, by prompt:
		 What kind of document is it?

Total retrieved documents: 6

Check retrieved document[0]:
 Policy
Mandatory
September  20
12
Issuing departement
Hum
an Resources
Target audience 
All
 employees
Approver
Executive Board, Nestlé S.A.
Repository
All Nestlé Principles and Policies, Standards and  
Guidelines can be found in the Centre online repository at:  
http://intranet.nestle.com/nestledocs
Copyright and confidentiality
Al
l rights belong to Nestec Ltd., Vevey, Switzerland.
© 2012, Nestec Ltd.
Design
Nestec Ltd., Corporate Identity & Design,  
Vevey, Switzerland
Production
brain’print GmbH, Switzerland
Paper
This report is printed on BVS, a paper produced  
from well-managed forests and other controlled sources  
certified by the Forest Stewardship Council (FSC).


In [6]:
#
# Generative AI Chat Model 
# 

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=500)

# Define the system prompt with placeholder for context
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n{context}"
)

# Build the chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


In [7]:
#
# Design Question Ansering Chain 
# 

# Create the document QA chain
question_answering_chain = create_stuff_documents_chain(llm, prompt)

# Combine retriever and QA chain (retriever should already be defined)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

# Use the RAG chain
user_input = "Help me understand the document"
response = rag_chain.invoke({"input": user_input})
print(response["answer"])




The document outlines Nestlé's Human Resources Policy, emphasizing the importance of employees in the company's success. It covers guidelines for effective HR management, the vision and mission of the HR function, and the Nestlé employee lifecycle. The policy focuses on putting people at the center of everything Nestlé does, promoting trust, respect, and a supportive work environment.


In [8]:
def tellme(prompt):
    response = rag_chain.invoke({"input": prompt})
    return(response["answer"])

In [9]:
tellme("what are rewards conditions")

"Total rewards at Nestlé include Fixed Pay, Variable Pay, Benefits, Personal Growth and Development, and Work Life Environment. These elements are designed to correspond to what is valued by employees in each market and demonstrate Nestlé's commitment to employee growth and contribution. The programs must be established within the social and legal framework of each country, respecting applicable collective agreements."

In [11]:
tellme("any policy for trainings")

'Nestlé has a comprehensive range of training activities and methodologies to support learning and growth for all employees. Training and development priorities are determined by the Company, with responsibility shared between employees, line managers, and Human Resources. Attending training programs is considered a component of ongoing development rather than a reward.'

In [12]:
tellme("what are leave policies")

"I don't have specific information on leave policies in the provided context."

In [13]:
tellme("how many hirerichial layers are theere")

'There are minimal levels of management mentioned in the context, indicating that Nestlé has fewer hierarchical layers to increase efficiency and flexibility within the organization. This approach allows for broad spans of control, promoting cooperation between colleagues and creating a more accountable and innovative company. The emphasis is on a dynamic organization that encourages multi-skilled teams and cooperation over internal competition.'

In [10]:
#
# Gradio APP for web-interface for testing 
#

import gradio as gr
import sys
gr.close_all() 

# Dummy response generator (replace with actual model logic)
def generate_answer(message, history):
    tellme(message)
    return tellme(message)

with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# Nestle's HR Policy")
    gr.Markdown("Ask any questions below:")

    chatbot = gr.Chatbot(height=300)
    message_input = gr.Textbox(placeholder="Ask any questions on HR policy", label="Your question")

    with gr.Row():
        ask_button = gr.Button("Ask")
        clear_button = gr.Button("Clear History")
        quit_button = gr.Button("Quit")

    history = []

    def respond(user_message, chat_history):
        response = generate_answer(user_message, chat_history)
        chat_history = chat_history + [(user_message, response)]
        return chat_history, ""

    def clear_chat():
        return [], ""

    def quit_app():
        sys.exit()

    ask_button.click(fn=respond, inputs=[message_input, chatbot], outputs=[chatbot, message_input])
    clear_button.click(fn=clear_chat, outputs=[chatbot, message_input])
    quit_button.click(fn=quit_app)

if __name__ == "__main__":
    demo.launch(share=True)


IMPORTANT: You are using gradio version 4.7.1, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861

Could not create share link. Missing file: /usr/local/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/site-packages/gradio
